In [2]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML # dùng để hiển thị 
%matplotlib inline 
import helper

#All the header files required for the code
from factor_analyzer import FactorAnalyzer
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import metrics
import random

In [3]:
#Importing both the file using pandas 
movies = pd.read_csv('movies data/movies data.csv')
ratings = pd.read_csv('movies data/ratings data.csv')
users = pd.read_csv('movies data/users data.csv')
#Deleting unnecessary columns
movies = movies.drop('Unnamed: 0',axis = 1)
ratings = ratings.drop(['Unnamed: 0','Timestamp'],axis = 1)
users = users.drop(['Unnamed: 0'],axis = 1)

print('Movies:',len(movies))
print('Ratings:',len(ratings))
print('Users:',len(users))

Movies: 3883
Ratings: 1000209
Users: 6040


In [4]:
ratings_mat = np.ndarray(
    shape=(np.max(ratings.MovieID.values), np.max(ratings.UserID.values)), dtype=np.uint8)
ratings_mat[ratings.MovieID.values-1, ratings.UserID.values-1] = ratings.Rating.values
print(ratings_mat)

[[  5   0   0 ...   0   0   3]
 [243   2   0 ...   0   0   0]
 [230   5   0 ...   0   0   0]
 ...
 [174   4   0 ...   0   0   0]
 [210   4   0 ...   0   0   0]
 [215  13   0 ...   0   0   0]]


In [5]:
normalised_mat = ratings_mat - np.asarray([(np.mean(ratings_mat, 1))]).T

In [29]:
A = normalised_mat.T / np.sqrt(ratings_mat.shape[0] - 1)
U, S, V = np.linalg.svd(A)

In [32]:
print(len(U))
print(len(S))
print(len(V))

6040
3952
3952


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
# user similarity on replacing NAN by item(movie) avg
cosine = cosine_similarity(final_movie)
np.fill_diagonal(cosine, 0 )
similarity_with_movie =pd.DataFrame(cosine,index=final_movie.index)
similarity_with_movie.columns=final_user.index
similarity_with_movie.head()

In [21]:
# def cosine_similarity(data, movie_id, top_n=10):
    

def top_cosine_similarity(data, movie_id, top_n=10):
    index = movie_id - 1 # Movie id starts from 1 in the dataset
    movie_row = data[index, :]    
    magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
    similarity = np.dot(movie_row, data.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity)
    return sort_indexes[:top_n]

# Function to print top N similar movies
def print_similar_movies(movie_data, movie_id, top_indexes):
    print('Recommendations for {0}: \n'.format(
    movie_data[movie_data.MovieID == movie_id].Title.values[0]))
    for id in top_indexes + 1:
        print(movie_data[movie_data.MovieID == id].Title.values[0])

In [34]:
#k-principal components to represent movies, movie_id to find recommendations, top_n print n results        
k = 3952
movie_id = 10 # (getting an id from movies.dat)
top_n = 10
sliced = V.T[:, :k] # representative data
indexes = top_cosine_similarity(sliced, movie_id, top_n)

#Printing the top N similar movies
print_similar_movies(movies, movie_id, indexes)

Recommendations for GoldenEye (1995): 

GoldenEye (1995)
Daylight (1996)
Heavy Metal (1981)
Night of the Living Dead (1968)
Dead Zone, The (1983)
Far and Away (1992)
Big (1988)
Three Colors: White (1994)
Bird on a Wire (1990)
Heaven & Earth (1993)


In [36]:
data = sliced

# chọn movie target
index = movie_id - 1 # Movie id starts from 1 in the dataset

movie_row = data[index, :]
magnitude = np.sqrt(np.einsum('ij, ij -> i', data, data))
similarity = np.dot(movie_row, data.T) / (magnitude[index] * magnitude)
print(similarity)
sort_indexes = np.argsort(-similarity)
print(sort_indexes)
# return sort_indexes[:top_n]


[ 7.91783725e-17 -1.77694346e-17  1.91133363e-16 ...  4.75472899e-17
  2.20363978e-17  9.39311838e-17]
[   9  797  609 ... 3073 1957 2288]
